# 目標：利用openpose和神經網路辨識T-POSE和DAB姿勢。

# 組員：
# 應數一 108701011 游能澤
# 應數一 108701034 柯里橫
# 應數一 108701018 池欣霓
# 應數一 108701019 許辰宇

# 分工：
# 構想與建設環境：游能澤、柯里橫
# 程式建構：柯里橫

# 由於一直無法使用pyopenpose函式，無法透過
# openpose取得樣本，只好找別人做好的。

# 到https://github.com/burningion/dab-and-tpose-controlled-lights/tree/master/data
# 載下作者生成好的數據，開個data資料夾放進去
# dabs.npy tposes.npy other.npy
# more-dabs.npy more-tposes.npy more-other.npy
# test-dabs.npy

In [1]:
import numpy as np

# 這幾個檔案是以numpy儲存的二進制文件
# 裡頭有我們需要的特徵

In [2]:
dabDataset = np.load('data/dabs.npy')
tposeDataset = np.load('data/tposes.npy')
otherDataset = np.load('data/other.npy')

In [3]:
dabDataset[0]

array([[5.8832416e+02, 2.9433704e+02, 7.2265184e-01],
       [5.8239331e+02, 3.5126093e+02, 8.0205584e-01],
       [5.0984329e+02, 3.4919385e+02, 7.5316119e-01],
       [4.1784265e+02, 3.1985785e+02, 8.1164622e-01],
       [3.6101605e+02, 2.9243521e+02, 8.0296052e-01],
       [6.5091376e+02, 3.6097537e+02, 6.4161348e-01],
       [6.3724268e+02, 2.7274924e+02, 7.8188539e-01],
       [4.9614203e+02, 2.4154723e+02, 8.3243752e-01],
       [5.4315808e+02, 6.4114813e+02, 4.4807938e-01],
       [4.8636816e+02, 6.2938318e+02, 3.6906898e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [6.0191382e+02, 6.4702966e+02, 3.8946095e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [5.7648334e+02, 2.7475522e+02, 6.1822432e-01],
       [6.0389270e+02, 2.8454663e+02, 4.1854110e-01],
       [5.5686536e+02, 2.6891223e+02, 2.7014270e-01],
       [6.1959991e+02, 2.924

In [4]:
dabDataset.shape

(14, 25, 3)

In [5]:
dabDataset[0].shape

(25, 3)

# Adding our Labels
# Our labels come from the [BODY_25 Pose Output format](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/output.md#pose-output-format-body_25) available at the repo.

In [6]:
labels = ["Nose", "Neck", "RShoulder", "RElbow", "RWrist", "LShoulder", "LElbow",
 "LWrist", "MidHip", "RHip", "RKnee", "RAnkle", "LHip", "LKnee", "LAnkle",
 "REye", "LEye", "REar", "LEar", "LBigToe", "LSmallToe", "LHeel", "RBigToe",
 "RSmallToe", "RHeel", "Background"]

## 看數據有三個維度分別是X、Ｙ、Confidence，不需要用到Confidence。

In [7]:
properLabels = []
for label in labels:
    properLabels.append(label + 'X')
    properLabels.append(label + 'Y')
    properLabels.append(label + 'Confidence')

In [8]:
import csv

with open('data/dabs.csv', 'w+') as dabcsv:
    dabwriter = csv.writer(dabcsv, delimiter=',')
    dabwriter.writerow(properLabels)
    for cell in dabDataset:
        dabwriter.writerow(cell.flatten())
        
with open('data/tposes.csv', 'w+') as tposecsv:
    tposewriter = csv.writer(tposecsv, delimiter=',')
    tposewriter.writerow(properLabels)
    for cell in tposeDataset:
        tposewriter.writerow(cell.flatten())
        
with open('data/other.csv', 'w+') as othercsv:
    otherwriter = csv.writer(othercsv, delimiter=',')
    otherwriter.writerow(properLabels)
    for cell in otherDataset:
        otherwriter.writerow(cell.flatten())

# 用CSV檔看訓練資料略少，但還是試著訓練看看。

# Creating a Labeled Dataset for Training and Testing
# We'll use 0 for other poses, 1 for dabs, and 2 for tposes.

In [9]:
labels = np.zeros(len(otherDataset))
labels = np.append(labels, np.full((len(dabDataset)), 1))
labels = np.append(labels, np.full((len(tposeDataset)), 2))
print(labels)
print("%i total examples for training." % len(labels))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2.]
56 total examples for training.


In [10]:
dataset = np.append(otherDataset, dabDataset, axis=0)
dataset = np.append(dataset, tposeDataset, axis=0)
print(dataset)

[[[488.3213     147.51425      0.83340967]
  [494.22372    284.5734       0.8012297 ]
  [386.4863     270.83716      0.66853976]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 [[515.7737     112.18818      0.83487195]
  [478.48004    274.7029       0.8005627 ]
  [368.77948    257.2105       0.6782713 ]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 [[547.1316     112.15151      0.79948723]
  [464.79065    268.88403      0.73338044]
  [360.98135    243.43745      0.62600124]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 ...

 [[509.97504    257.06958      0.892523  ]
  [460.9663     351.17117      0.7867987 ]
  [372.75305    333.54434      0.6111988 ]
  ...
  [  0.           0.           0.        ]
  [  0.           

In [11]:
dataset.shape

(56, 25, 3)

## 讓數值變成0~1之間

In [12]:
dataset[:,:,1] / 1280

array([[0.11524551, 0.22232297, 0.21159153, ..., 0.        , 0.        ,
        0.        ],
       [0.08764701, 0.21461165, 0.2009457 , ..., 0.        , 0.        ,
        0.        ],
       [0.08761837, 0.21006565, 0.19018552, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.20083562, 0.2743525 , 0.26058152, ..., 0.        , 0.        ,
        0.        ],
       [0.2238348 , 0.29272196, 0.27588812, ..., 0.        , 0.        ,
        0.        ],
       [0.2100658 , 0.28972745, 0.2758514 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

# Shuffle labels and features

In [13]:
from sklearn.utils import shuffle
X, y = shuffle(dataset, labels)
from keras.utils.np_utils import to_categorical
y = to_categorical(y, 3)
print(y.shape[1])

Using TensorFlow backend.


3


# 架設神經網路

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD

In [15]:
X = X.reshape(len(X), 75)
dataset[0]

array([[4.88321289e+02, 1.47514252e+02, 8.33409667e-01],
       [4.94223724e+02, 2.84573395e+02, 8.01229715e-01],
       [3.86486298e+02, 2.70837158e+02, 6.68539762e-01],
       [3.37498718e+02, 4.31440033e+02, 8.06459844e-01],
       [2.76727325e+02, 5.92155334e+02, 6.95721209e-01],
       [6.01926575e+02, 2.96297577e+02, 6.77372575e-01],
       [6.17621460e+02, 4.47154175e+02, 8.15527081e-01],
       [6.33207092e+02, 6.13721497e+02, 7.50288665e-01],
       [4.49166534e+02, 6.23515259e+02, 3.33123893e-01],
       [3.68768433e+02, 6.15664124e+02, 2.96909660e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.29464417e+02, 6.35266663e+02, 3.00662249e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.72626495e+02, 1.25848732e+02, 7.93599010e-01],
       [5.09897217e+02, 1.25915306e+02, 8.75047982e-01],
       [4.47158661e+02, 1.31820

In [16]:
model = Sequential()
model.add(Dense(128, activation = 'relu', input_shape = (75,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(optimizer = SGD(lr = 0.005),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.fit(X, y, epochs = 200, batch_size = 64)

Epoch 1/200
56/56 [==============================] - 0s 3ms/step - loss: 349.1770 - accuracy: 0.3571
Epoch 2/200
56/56 [==============================] - 0s 53us/step - loss: 10615.8975 - accuracy: 0.2500
Epoch 3/200
56/56 [==============================] - 0s 35us/step - loss: 40285.3789 - accuracy: 0.4643
Epoch 4/200
56/56 [==============================] - 0s 32us/step - loss: 39818.2070 - accuracy: 0.2857
Epoch 5/200
56/56 [==============================] - 0s 66us/step - loss: 6458.7578 - accuracy: 0.4643
Epoch 6/200
56/56 [==============================] - 0s 85us/step - loss: 1.0930 - accuracy: 0.3393
Epoch 7/200
56/56 [==============================] - 0s 67us/step - loss: 1.0814 - accuracy: 0.4643
Epoch 8/200
56/56 [==============================] - 0s 40us/step - loss: 1.1014 - accuracy: 0.4643
Epoch 9/200
56/56 [==============================] - 0s 39us/step - loss: 1.0853 - accuracy: 0.4643
Epoch 10/200
56/56 [==============================] - 0s 37us/step - loss: 1.0844 - 

56/56 [==============================] - 0s 43us/step - loss: 1.1028 - accuracy: 0.4643
Epoch 83/200
56/56 [==============================] - 0s 38us/step - loss: 1.0535 - accuracy: 0.4643
Epoch 84/200
56/56 [==============================] - 0s 50us/step - loss: 1.0909 - accuracy: 0.4643
Epoch 85/200
56/56 [==============================] - 0s 41us/step - loss: 1.0707 - accuracy: 0.4643
Epoch 86/200
56/56 [==============================] - 0s 39us/step - loss: 1.0798 - accuracy: 0.4643
Epoch 87/200
56/56 [==============================] - 0s 41us/step - loss: 1.0680 - accuracy: 0.4643
Epoch 88/200
56/56 [==============================] - 0s 35us/step - loss: 1.0735 - accuracy: 0.4643
Epoch 89/200
56/56 [==============================] - 0s 40us/step - loss: 1.1026 - accuracy: 0.4643
Epoch 90/200
56/56 [==============================] - 0s 46us/step - loss: 1.0710 - accuracy: 0.4643
Epoch 91/200
56/56 [==============================] - 0s 34us/step - loss: 1.0706 - accuracy: 0.4643
Epo

Epoch 163/200
56/56 [==============================] - 0s 51us/step - loss: 1.0867 - accuracy: 0.4643
Epoch 164/200
56/56 [==============================] - 0s 41us/step - loss: 1.0703 - accuracy: 0.4643
Epoch 165/200
56/56 [==============================] - 0s 56us/step - loss: 1.0695 - accuracy: 0.4643
Epoch 166/200
56/56 [==============================] - 0s 41us/step - loss: 1.0898 - accuracy: 0.4643
Epoch 167/200
56/56 [==============================] - 0s 49us/step - loss: 1.0861 - accuracy: 0.4643
Epoch 168/200
56/56 [==============================] - 0s 55us/step - loss: 1.0661 - accuracy: 0.4643
Epoch 169/200
56/56 [==============================] - 0s 44us/step - loss: 1.0770 - accuracy: 0.4643
Epoch 170/200
56/56 [==============================] - 0s 50us/step - loss: 1.0834 - accuracy: 0.4643
Epoch 171/200
56/56 [==============================] - 0s 63us/step - loss: 1.0667 - accuracy: 0.4643
Epoch 172/200
56/56 [==============================] - 0s 65us/step - loss: 1.0761

# Cleaning up data further¶

## accuracy too low

In [17]:
X, y = shuffle(dataset, labels)
y = to_categorical(y, 3)
print(X.shape)
X[:,:,0] = X[:,:,0] / 720 
X[:,:,1] = X[:,:,1] / 1280
X = X[:,:,:2]
print(X.shape)
X = X.reshape(56, 50)     
print(X.shape)

(56, 25, 3)
(56, 25, 2)
(56, 50)


In [18]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(50,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=200,batch_size=64)

Epoch 1/200
56/56 [==============================] - 0s 3ms/step - loss: 1.0406 - accuracy: 0.4821
Epoch 2/200
56/56 [==============================] - 0s 36us/step - loss: 1.1201 - accuracy: 0.4107
Epoch 3/200
56/56 [==============================] - 0s 59us/step - loss: 1.0904 - accuracy: 0.3750
Epoch 4/200
56/56 [==============================] - 0s 52us/step - loss: 1.1830 - accuracy: 0.3036
Epoch 5/200
56/56 [==============================] - 0s 39us/step - loss: 1.1125 - accuracy: 0.3214
Epoch 6/200
56/56 [==============================] - 0s 36us/step - loss: 1.0818 - accuracy: 0.5000
Epoch 7/200
56/56 [==============================] - 0s 72us/step - loss: 1.1210 - accuracy: 0.4643
Epoch 8/200
56/56 [==============================] - 0s 59us/step - loss: 1.1126 - accuracy: 0.4107
Epoch 9/200
56/56 [==============================] - 0s 49us/step - loss: 1.1226 - accuracy: 0.4464
Epoch 10/200
56/56 [==============================] - 0s 61us/step - loss: 1.0506 - accuracy: 0.5000


Epoch 83/200
56/56 [==============================] - 0s 52us/step - loss: 0.6506 - accuracy: 0.8036
Epoch 84/200
56/56 [==============================] - 0s 42us/step - loss: 0.6389 - accuracy: 0.7143
Epoch 85/200
56/56 [==============================] - 0s 55us/step - loss: 0.5546 - accuracy: 0.8750
Epoch 86/200
56/56 [==============================] - 0s 43us/step - loss: 0.6370 - accuracy: 0.7321
Epoch 87/200
56/56 [==============================] - 0s 44us/step - loss: 0.6159 - accuracy: 0.8036
Epoch 88/200
56/56 [==============================] - 0s 49us/step - loss: 0.5777 - accuracy: 0.7679
Epoch 89/200
56/56 [==============================] - 0s 53us/step - loss: 0.6161 - accuracy: 0.7500
Epoch 90/200
56/56 [==============================] - 0s 43us/step - loss: 0.6721 - accuracy: 0.7321
Epoch 91/200
56/56 [==============================] - 0s 55us/step - loss: 0.6290 - accuracy: 0.6964
Epoch 92/200
56/56 [==============================] - 0s 46us/step - loss: 0.5752 - accurac

Epoch 164/200
56/56 [==============================] - 0s 35us/step - loss: 0.4008 - accuracy: 0.8036
Epoch 165/200
56/56 [==============================] - 0s 40us/step - loss: 0.2823 - accuracy: 0.8929
Epoch 166/200
56/56 [==============================] - 0s 38us/step - loss: 0.3017 - accuracy: 0.8750
Epoch 167/200
56/56 [==============================] - 0s 34us/step - loss: 0.3650 - accuracy: 0.8571
Epoch 168/200
56/56 [==============================] - 0s 36us/step - loss: 0.3413 - accuracy: 0.8750
Epoch 169/200
56/56 [==============================] - 0s 45us/step - loss: 0.3408 - accuracy: 0.8571
Epoch 170/200
56/56 [==============================] - 0s 51us/step - loss: 0.2701 - accuracy: 0.8929
Epoch 171/200
56/56 [==============================] - 0s 55us/step - loss: 0.3290 - accuracy: 0.8393
Epoch 172/200
56/56 [==============================] - 0s 36us/step - loss: 0.3566 - accuracy: 0.8214
Epoch 173/200
56/56 [==============================] - 0s 50us/step - loss: 0.2628

# Adding More Data and Beginning Data Augmentation

In [19]:
dabDataset = np.load('data/more-dabs.npy')
tposeDataset = np.load('data/more-tposes.npy')
otherDataset = np.load('data/more-other.npy')
labels1 = np.zeros(len(otherDataset))
labels1 = np.append(labels1, np.full((len(dabDataset)), 1))
labels1 = np.append(labels1, np.full((len(tposeDataset)), 2))
print(labels1)
print("%i total new samples" % len(labels1))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2.]
171 total new samples


In [20]:
dataset1 = np.append(otherDataset, dabDataset, axis=0)
dataset1 = np.append(dataset1, tposeDataset, axis=0)
X1, y1 = shuffle(dataset1, labels1)
y1 = to_categorical(y1, 3)
print(X1.shape)
X1[:,:,0] = X1[:,:,0] / 720
X1[:,:,1] = X1[:,:,1] / 1280
X1 = X1[:,:,:2]
print(X1.shape)
X1 = X1.reshape(len(X1), 50)    
print(X1.shape)

(171, 25, 3)
(171, 25, 2)
(171, 50)


In [21]:
model = Sequential()
model.add(Dense(128, activation = 'relu', input_shape = (50,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(optimizer = 'Adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
history = model.fit(X1, y1, epochs = 200, batch_size = 64)

Epoch 1/200
171/171 [==============================] - 0s 817us/step - loss: 1.1506 - accuracy: 0.3977
Epoch 2/200
171/171 [==============================] - 0s 41us/step - loss: 1.1650 - accuracy: 0.3801
Epoch 3/200
171/171 [==============================] - 0s 40us/step - loss: 1.1043 - accuracy: 0.3918
Epoch 4/200
171/171 [==============================] - 0s 44us/step - loss: 1.1017 - accuracy: 0.3977
Epoch 5/200
171/171 [==============================] - 0s 41us/step - loss: 1.0634 - accuracy: 0.3801
Epoch 6/200
171/171 [==============================] - 0s 41us/step - loss: 0.9900 - accuracy: 0.5380
Epoch 7/200
171/171 [==============================] - 0s 45us/step - loss: 0.9947 - accuracy: 0.4912
Epoch 8/200
171/171 [==============================] - 0s 60us/step - loss: 1.0302 - accuracy: 0.4503
Epoch 9/200
171/171 [==============================] - 0s 50us/step - loss: 0.9533 - accuracy: 0.5906
Epoch 10/200
171/171 [==============================] - 0s 49us/step - loss: 0.98

171/171 [==============================] - 0s 51us/step - loss: 0.2204 - accuracy: 0.9181
Epoch 81/200
171/171 [==============================] - 0s 60us/step - loss: 0.2210 - accuracy: 0.9240
Epoch 82/200
171/171 [==============================] - 0s 45us/step - loss: 0.2416 - accuracy: 0.9123
Epoch 83/200
171/171 [==============================] - 0s 51us/step - loss: 0.2020 - accuracy: 0.9591
Epoch 84/200
171/171 [==============================] - 0s 50us/step - loss: 0.1638 - accuracy: 0.9532
Epoch 85/200
171/171 [==============================] - 0s 44us/step - loss: 0.1710 - accuracy: 0.9357
Epoch 86/200
171/171 [==============================] - 0s 44us/step - loss: 0.2051 - accuracy: 0.9532
Epoch 87/200
171/171 [==============================] - 0s 45us/step - loss: 0.1453 - accuracy: 0.9591
Epoch 88/200
171/171 [==============================] - 0s 49us/step - loss: 0.1965 - accuracy: 0.9415
Epoch 89/200
171/171 [==============================] - 0s 44us/step - loss: 0.1748 - 

171/171 [==============================] - 0s 39us/step - loss: 0.0680 - accuracy: 0.9825
Epoch 160/200
171/171 [==============================] - 0s 40us/step - loss: 0.0566 - accuracy: 0.9825
Epoch 161/200
171/171 [==============================] - 0s 40us/step - loss: 0.0416 - accuracy: 0.9942
Epoch 162/200
171/171 [==============================] - 0s 39us/step - loss: 0.0758 - accuracy: 0.9766
Epoch 163/200
171/171 [==============================] - 0s 36us/step - loss: 0.0589 - accuracy: 0.9766
Epoch 164/200
171/171 [==============================] - 0s 39us/step - loss: 0.0625 - accuracy: 0.9649
Epoch 165/200
171/171 [==============================] - 0s 40us/step - loss: 0.0549 - accuracy: 0.9766
Epoch 166/200
171/171 [==============================] - 0s 40us/step - loss: 0.0321 - accuracy: 1.0000
Epoch 167/200
171/171 [==============================] - 0s 36us/step - loss: 0.0453 - accuracy: 1.0000
Epoch 168/200
171/171 [==============================] - 0s 37us/step - loss: 

In [22]:
model.predict_classes(X)

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
X

array([[0.59924173, 0.06160351, 0.7543989 , ..., 0.        , 0.        ,
        0.        ],
       [0.71635234, 0.08764701, 0.6645556 , ..., 0.        , 0.        ,
        0.        ],
       [0.7708483 , 0.18869254, 0.7353909 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.66201735, 0.12128913, 0.6184485 , ..., 0.        , 0.        ,
        0.        ],
       [0.6619014 , 0.12130284, 0.6184055 , ..., 0.        , 0.        ,
        0.        ],
       [0.8197582 , 0.11365898, 0.6617935 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [24]:
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0

In [25]:
model.test_on_batch(X, y)

[4.4658914, 0.8722467]

In [26]:
model.save('data/dab-tpose-other.h5')

In [27]:
import keras
modello = keras.models.load_model('data/dab-tpose-other.h5')

In [28]:
dabDataset = np.load('data/test-dabs.npy')
dabDataset[:,:,0] = dabDataset[:,:,0] / 720
dabDataset[:,:,1] = dabDataset[:,:,1] / 1280
dabDataset = dabDataset[:,:,:2]
dabDataset = dabDataset.reshape(len(dabDataset), 50)
modello.predict_classes(dabDataset)

array([1, 1, 1, 1, 1, 1])

# 預測結果全是1，從前面我們設定的數字，1代表dabs，預測成功。

# 關於專案分工的部分預計資料前處理和使用模型交給另外兩個人，但費勁了洪荒之力還是做不出來，最後放棄做那些部分，剩下上訓練模型的部分。

# 作者在他自己的blog上發這篇（https://www.makeartwithpython.com/blog/dab-and-tpose-controlled-lights/，裡頭提到了「All of the code, models, and training data are freely available on Github.」，可能在是否能公開發表上有模擬地帶，下次會注意授權問題，並且先寄信詢問。
# 模型是做好了，但卻無法使用，由於導入pyopenpose的不成功，無法在python裡呼叫openpose來取得測試資料，沒辦法實際應用。